<table style="float:left; border:none">
   <tr style="border:none; background-color: #ffffff">
       <td style="border:none">
           <a href="http://bokeh.pydata.org/">     
           <img 
               src="assets/bokeh-transparent.png" 
               style="width:50px"
           >
           </a>    
       </td>
       <td style="border:none">
           <h1>Bokeh 教程</h1>
       </td>
   </tr>
</table>

<div style="float:right;"><h2>11. 运行 Bokeh 应用</h2></div>

Bokeh的架构是这样的，高层的“模型对象”（表示plots, ranges, axes, glyphs等）使用Python创建，然后转换成JSON格式传递给客户端库BokehJS。使用Bokeh服务器，则能够保持“模型对象”在Python和浏览器间彼此同步，创造强大的能力：

* 使用Python的强劲功能进行计算或查询来响应浏览器中的UI和工具事件
* 自动推送更新浏览器中的UI（即小部件或图表）
* 使用周期，超时，和异步回调来驱动无缝更新 

***这种 Python 和 浏览器 之间的同步能力是 Bokeh server 的主要目的。***

<center><div style="font-size: 18pt;color: firebrick;"> 注意：以下作业需要在notebook之外的工作 <div></center>

# Hello Bokeh

要尝试下面的示例，将代码复制到文件 ``hello.py`` 中，然后执行：
```bash
bokeh serve --show hello.py 
```
----

```python
# hello.py 

from bokeh.io import curdoc
from bokeh.layouts import column
from bokeh.models.widgets import TextInput, Button, Paragraph

# create some widgets
button = Button(label="Say HI")
input = TextInput(value="Bokeh")
output = Paragraph()

# add a callback to a widget
def update():
    output.text = "Hello, " + input.value
button.on_click(update)

# create a layout for everything
layout = column(button, input, output)

# add the layout to curdoc
curdoc().add_root(layout)
```

让我们尝试一个练习：修改这个示例来添加另一个控件。

In [1]:
# EXERCISE: add a Select widget to this example that offers several different greetings



# 连接Plots和Widgets

要尝试下面的例子，将代码复制到一个文件 ``app.py`` 中，然后执行：
```bash
bokeh serve --show app.py
```
----

```python
# app.py

from numpy.random import random

from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.plotting import ColumnDataSource, Figure
from bokeh.models.widgets import Select, TextInput

def get_data(N):
    return dict(x=random(size=N), y=random(size=N), r=random(size=N) * 0.03)

source = ColumnDataSource(data=get_data(200))

p = Figure(tools="", toolbar_location=None)
r = p.circle(x='x', y='y', radius='r', source=source,
             color="navy", alpha=0.6, line_color="white")

COLORS = ["black", "firebrick", "navy", "olive", "goldenrod"]
select = Select(title="Color", value="navy", options=COLORS)
input = TextInput(title="Number of points", value="200")

def update_color(attrname, old, new):
    r.glyph.fill_color = select.value
select.on_change('value', update_color)

def update_points(attrname, old, new):
    N = int(input.value)
    source.data = get_data(N)
input.on_change('value', update_points)

layout = column(row(select, input, width=400), row(p))

curdoc().add_root(layout)
```

In [2]:
# EXERCISE: add more widgets to change more aspects of this plot


# Streaming Data

通过使用 ``stream`` 方法，可以高效地将新数据导流到列数据源。这个方法接受两个参数：
* ``new_data`` &mdash; 与列数据源结构相同的字典
* ``rollover`` &mdash; client上的最大列长度（较早的数据被丢弃） *[可选]*

如果未指定 ``rollover``，则在client上的数据不会丢弃，而列的增长没有限制。

导流数据结合周期性回调函数通常非常有用。``curdoc()`` 的 ``add_periodic_callback`` 方法接受一个回调函数，和一个时间间隔（毫秒）以重复执行此回调函数。

要尝试下面的示例，请将代码复制到一个文件 ``stream.py`` 中，然后执行：
```bash
bokeh serve --show stream.py
```
----

```python
# stream.py
from math import cos, sin

from bokeh.io import curdoc
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

p = figure(x_range=(-1.1, 1.1), y_range=(-1.1, 1.1))
p.circle(x=0, y=0, radius=1, fill_color=None, line_width=2)

# this is the data source we will stream to
source = ColumnDataSource(data=dict(x=[1], y=[0]))
p.circle(x='x', y='y', size=12, fill_color='white', source=source)

def update():
    x, y = source.data['x'][-1], source.data['y'][-1]
  
    # construct the new values for all columns, and pass to stream
    new_data = dict(x=[x*cos(0.1) - y*sin(0.1)], y=[x*sin(0.1) + y*cos(0.1)])
    source.stream(new_data, rollover=8)

curdoc().add_periodic_callback(update, 150)
curdoc().add_root(p)
```

In [3]:
### EXERCISE: starting with the above example, create your own streaming plot
